In [1]:
import os
import glob
from langchain_community.document_loaders import PyPDFLoader

# Chemin vers votre dossier contenant les PDF
pdf_folder = r"C:\Users\Mazen ISMAIL\Downloads\Rag project\Data"

# Récupérer tous les fichiers PDF dans le dossier
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))
print(f"Nombre de fichiers PDF trouvés : {len(pdf_files)}")

# Afficher les noms des fichiers trouvés
for pdf in pdf_files:
    print(f"- {os.path.basename(pdf)}")

Nombre de fichiers PDF trouvés : 13
- BASF_sustainability_report_2024.pdf
- Bayer_sustainability_report_2024.pdf
- Cargill_sustainability_report_2024.pdf
- CK_Hutchinson _sustainability_report_2024.pdf
- Honeywell_sustainability_report_2024.pdf
- Merck_sustainability_report_2024.pdf
- Microsoft_sustainability_report_2024.pdf
- Qualcomm_sustainability_report_2023.pdf
- Shell_sustainability_report_2023.pdf
- Thai_Oil_sustainability_report_2023.pdf
- Veolia_sustainability_report_2024.pdf
- Verizon_sustainability_report_2023.pdf
- Walmart_sustainability_report_2023.pdf


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.schema import Document
import PyPDF2
import os
import glob

# Initialiser le text splitter pour un chunking plus fin et hiérarchique
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,  # Taille réduite comme demandé
    chunk_overlap=50,  # Chevauchement adapté à la nouvelle taille
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Séparateurs plus précis
)

pdf_folder = r"C:\Users\Mazen ISMAIL\Downloads\Rag project\Data"
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

# Charger et découper les PDF un par un avec support des tableaux et images
all_chunks = []
for pdf_file in pdf_files:
    try:
        print(f"Traitement de {os.path.basename(pdf_file)}...")
        
        # Utiliser PyPDF2 pour extraire le texte
        with open(pdf_file, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            pdf_text = ""
            
            # Métadonnées du document
            doc_metadata = {
                "source": pdf_file,
                "title": os.path.basename(pdf_file),
                "total_pages": len(reader.pages)
            }
            
            for page_num, page in enumerate(reader.pages):
                # Extraire le texte
                text = page.extract_text() or ""
                
                # Combiner avec des marqueurs de structure
                page_content = f"# Page {page_num+1}\n{text}"
                pdf_text += page_content
        
        # Découper avec préservation de la structure
        try:
            header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
                ("#", "header1"),
                ("##", "header2"),
            ])
            md_header_splits = header_splitter.split_text(pdf_text)
            
            # Découper en chunks tout en préservant les métadonnées hiérarchiques
            chunks = []
            for split in md_header_splits:
                # Préserver le contexte hiérarchique dans les métadonnées
                split_metadata = {**doc_metadata, **split.metadata}
                # Découper davantage si nécessaire
                smaller_chunks = text_splitter.split_text(split.page_content)
                # Créer des Documents avec contexte préservé
                for chunk in smaller_chunks:
                    chunks.append(Document(page_content=chunk, metadata=split_metadata))
        except Exception as e:
            print(f"Erreur lors du découpage hiérarchique: {e}")
            # Fallback au découpage simple
            chunks = []
            with open(pdf_file, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for i, page in enumerate(reader.pages):
                    text = page.extract_text() or ""
                    page_doc = Document(
                        page_content=text,
                        metadata={"source": pdf_file, "page": i+1}
                    )
                    page_chunks = text_splitter.split_documents([page_doc])
                    chunks.extend(page_chunks)
        
        all_chunks.extend(chunks)
        print(f"  - {len(chunks)} chunks créés")
    except Exception as e:
        print(f"Erreur avec {os.path.basename(pdf_file)}: {e}")

print(f"Total : {len(all_chunks)} chunks créés")

Traitement de BASF_sustainability_report_2024.pdf...
  - 2064 chunks créés
Traitement de Bayer_sustainability_report_2024.pdf...
Erreur avec Bayer_sustainability_report_2024.pdf: PyCryptodome is required for AES algorithm
Traitement de Cargill_sustainability_report_2024.pdf...
  - 968 chunks créés
Traitement de CK_Hutchinson _sustainability_report_2024.pdf...
  - 1119 chunks créés
Traitement de Honeywell_sustainability_report_2024.pdf...
  - 621 chunks créés
Traitement de Merck_sustainability_report_2024.pdf...
  - 1465 chunks créés
Traitement de Microsoft_sustainability_report_2024.pdf...
  - 1115 chunks créés
Traitement de Qualcomm_sustainability_report_2023.pdf...
  - 831 chunks créés
Traitement de Shell_sustainability_report_2023.pdf...
  - 1177 chunks créés
Traitement de Thai_Oil_sustainability_report_2023.pdf...
  - 803 chunks créés
Traitement de Veolia_sustainability_report_2024.pdf...
  - 392 chunks créés
Traitement de Verizon_sustainability_report_2023.pdf...
  - 924 chunks cr

In [3]:
from langchain_openai import OpenAIEmbeddings
import os

from dotenv import load_dotenv
load_dotenv(dotenv_path="OpenAIKey.env")

# Créer l'instance d'embeddings
try:
    # Pour GPT-4o mini, utilisez le modèle d'embedding approprié 
    # (text-embedding-3-small est un bon choix économique)
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # Test pour vérifier que ça fonctionne
    if all_chunks:
        test_text = all_chunks[0].page_content[:100]
        test_embedding = embeddings.embed_query(test_text)
        print(f"✅ Embedding créé avec succès - Dimension: {len(test_embedding)}")
except Exception as e:
    print(f"❌ Erreur lors de la création des embeddings: {e}")

✅ Embedding créé avec succès - Dimension: 1536


In [4]:
# Fonction pour traiter les chunks par lots
def process_chunks_in_batches(chunks, batch_size=100):
    """Traite les chunks par lots pour éviter les limitations d'API"""
    total_chunks = len(chunks)
    results = []
    
    for i in range(0, total_chunks, batch_size):
        batch = chunks[i:min(i + batch_size, total_chunks)]
        print(f"Traitement du lot {i//batch_size + 1}/{(total_chunks-1)//batch_size + 1}...")
        
        # Extraire le texte de chaque chunk pour l'embedding
        texts = [chunk.page_content for chunk in batch]
        
        try:
            # Créer les embeddings pour ce lot
            batch_embeddings = embeddings.embed_documents(texts)
            
            # Stocker les résultats (texte, embedding, métadonnées)
            for j, embedding_vector in enumerate(batch_embeddings):
                chunk_data = {
                    "text": batch[j].page_content,
                    "embedding": embedding_vector,
                    "metadata": batch[j].metadata
                }
                results.append(chunk_data)
                
        except Exception as e:
            print(f"Erreur lors du traitement du lot {i//batch_size + 1}: {e}")
    
    return results

# Traiter tous les chunks
print("Création des embeddings pour tous les chunks...")
embedded_chunks = process_chunks_in_batches(all_chunks, batch_size=50)
print(f"✅ Embeddings créés pour {len(embedded_chunks)} chunks sur {len(all_chunks)}")

Création des embeddings pour tous les chunks...
Traitement du lot 1/236...
Traitement du lot 2/236...
Traitement du lot 3/236...
Traitement du lot 4/236...
Traitement du lot 5/236...
Traitement du lot 6/236...
Traitement du lot 7/236...
Traitement du lot 8/236...
Traitement du lot 9/236...
Traitement du lot 10/236...
Traitement du lot 11/236...
Traitement du lot 12/236...
Traitement du lot 13/236...
Traitement du lot 14/236...
Traitement du lot 15/236...
Traitement du lot 16/236...
Traitement du lot 17/236...
Traitement du lot 18/236...
Traitement du lot 19/236...
Traitement du lot 20/236...
Traitement du lot 21/236...
Traitement du lot 22/236...
Traitement du lot 23/236...
Traitement du lot 24/236...
Traitement du lot 25/236...
Traitement du lot 26/236...
Traitement du lot 27/236...
Traitement du lot 28/236...
Traitement du lot 29/236...
Traitement du lot 30/236...
Traitement du lot 31/236...
Traitement du lot 32/236...
Traitement du lot 33/236...
Traitement du lot 34/236...
Traitemen

In [5]:
from langchain_community.vectorstores import FAISS
import pickle
import os

# Créer un répertoire pour stocker l'index FAISS si nécessaire
index_folder = os.path.join(os.path.dirname(pdf_folder), "faiss_index")
os.makedirs(index_folder, exist_ok=True)

# Chemin du fichier de sauvegarde
index_file_path = os.path.join(index_folder, "pdf_embeddings_index.faiss")
index_pkl_path = os.path.join(index_folder, "pdf_embeddings_index.pkl")

# Fonction pour créer et sauvegarder l'index FAISS
def create_and_save_faiss_index(embedded_chunks, embeddings_model):
    # Préparer les données pour FAISS
    texts = [item["text"] for item in embedded_chunks]
    embeddings_list = [item["embedding"] for item in embedded_chunks]
    metadatas = [item["metadata"] for item in embedded_chunks]
    
    # Créer l'index FAISS
    print("Création de l'index FAISS...")
    vector_store = FAISS.from_embeddings(
        text_embeddings=list(zip(texts, embeddings_list)),
        embedding=embeddings_model,
        metadatas=metadatas
    )
    
    # Sauvegarder l'index
    print(f"Sauvegarde de l'index FAISS dans {index_folder}...")
    vector_store.save_local(index_folder)
    
    print("✅ Index FAISS créé et sauvegardé avec succès!")
    return vector_store

# Fonction pour charger l'index FAISS s'il existe déjà
def load_faiss_index(embeddings_model):
    if os.path.exists(os.path.join(index_folder, "index.faiss")):
        print("Chargement de l'index FAISS existant...")
        try:
            vector_store = FAISS.load_local(index_folder, embeddings_model)
            print("✅ Index FAISS chargé avec succès!")
            return vector_store
        except Exception as e:
            print(f"Erreur lors du chargement de l'index: {e}")
    
    print("❌ Aucun index FAISS existant trouvé.")
    return None

# Créer ou charger l'index FAISS
vector_store = load_faiss_index(embeddings)
if vector_store is None:
    vector_store = create_and_save_faiss_index(embedded_chunks, embeddings)

# Vérifier que l'index fonctionne
test_query = "Qu'est-ce que le RAG?"
print(f"Test de recherche pour: '{test_query}'")
test_results = vector_store.similarity_search(test_query, k=2)
print(f"Trouvé {len(test_results)} résultats pertinents.")
if test_results:
    print(f"Exemple de résultat: {test_results[0].page_content[:150]}...")

❌ Aucun index FAISS existant trouvé.
Création de l'index FAISS...
Sauvegarde de l'index FAISS dans C:\Users\Mazen ISMAIL\Downloads\Rag project\faiss_index...
✅ Index FAISS créé et sauvegardé avec succès!
Test de recherche pour: 'Qu'est-ce que le RAG?'
Trouvé 2 résultats pertinents.
Exemple de résultat: procedure (e-Rapid Incident Report). The aim is to identify risks at an early stage and, if necessary,
initiate appropriate remedial and communication...


In [6]:
# Fonction modifiée pour charger l'index FAISS avec l'option de sécurité activée
def load_faiss_index(embeddings_model):
    if os.path.exists(os.path.join(index_folder, "index.faiss")):
        print("Chargement de l'index FAISS existant...")
        try:
            vector_store = FAISS.load_local(
                index_folder, 
                embeddings_model,
                allow_dangerous_deserialization=True  # Ajout de cette option
            )
            print("✅ Index FAISS chargé avec succès!")
            return vector_store
        except Exception as e:
            print(f"Erreur lors du chargement de l'index: {e}")
    
    print("❌ Aucun index FAISS existant trouvé.")
    return None

In [7]:
import streamlit as st
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.schema import Document
import fitz  # PyMuPDF
import tempfile
import shutil
import time
from dotenv import load_dotenv

# Chargement des variables d'environnement
load_dotenv(dotenv_path="OpenAIKey.env")

# Configuration de la page Streamlit avec thème
st.set_page_config(
    page_title="Assistant RAG pour PDF",
    page_icon="📚",
    layout="wide",
    initial_sidebar_state="expanded"
)

2025-05-18 20:53:11.057 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
# Appliquer des styles CSS améliorés
st.markdown("""
<style>
    /* Styles pour les titres et sous-titres */
    .main-header {
        font-size: 2.5rem;
        font-weight: 700;
        color: #1E88E5;
        margin-bottom: 1.5rem;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .sub-header {
        font-size: 1.8rem;
        font-weight: 600;
        color: #0D47A1;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        margin-top: 1.5rem;
        margin-bottom: 1rem;
    }
    
    /* Box d'information */
    .info-box {
        background-color: #E3F2FD;
        padding: 1.2rem;
        border-radius: 0.7rem;
        margin-bottom: 1.5rem;
        border-left: 5px solid #1E88E5;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    
    /* Conteneur de résultat */
    .result-container {
        background-color: #FAFAFA;
        padding: 1.8rem;
        border-radius: 0.7rem;
        border-left: 7px solid #1E88E5;
        margin-top: 1.5rem;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
    }
    
    /* Titre de source */
    .source-title {
        font-weight: 600;
        color: #0D47A1;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    
    /* Section de barre latérale */
    .sidebar-section {
        background-color: #F5F5F5;
        padding: 1.2rem;
        border-radius: 0.7rem;
        margin-bottom: 1.5rem;
        border-left: 4px solid #1E88E5;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    
    /* Style pour les tabs */
    .stTabs [data-baseweb="tab-list"] {
        gap: 2rem;
    }
    
    .stTabs [data-baseweb="tab"] {
        height: 3rem;
        white-space: pre-wrap;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        font-weight: 500;
    }
    
    /* Tableaux de données */
    .dataframe-container {
        border-radius: 0.7rem;
        overflow: hidden;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
        margin-top: 1rem;
    }
    
    /* Boutons */
    .stButton > button {
        font-weight: 600;
        padding: 0.6rem 1.2rem;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    
    /* Uniformisation des polices pour tout le contenu */
    body, .stTextInput label, .stTextInput input, .stTextArea label, .stTextArea textarea,
    .stSelectbox label, .stSelectbox select, .stSlider label, p, div, span {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
    }
    
    /* Style pour les métriques */
    .metric-container {
        background-color: #F5F5F5;
        padding: 1rem;
        border-radius: 0.5rem;
        text-align: center;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05);
    }
    
    .metric-value {
        font-size: 1.8rem;
        font-weight: 700;
        color: #1E88E5;
    }
    
    .metric-label {
        font-size: 1rem;
        color: #555;
        margin-top: 0.3rem;
    }
    
    /* Style pour les onglets principaux */
    .main-tabs {
        margin-top: 1.5rem;
    }
</style>
""", unsafe_allow_html=True)

2025-05-18 20:53:16.738 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:17.028 
  command:

    streamlit run c:\Users\Mazen ISMAIL\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-18 20:53:17.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [19]:
# Chemin vers l'index FAISS préexistant
index_folder = r"C:\Users\Mazen ISMAIL\Downloads\Rag project\faiss_index"

# Titre et description
st.markdown('<div class="main-header">📚 Assistant RAG pour PDF</div>', unsafe_allow_html=True)
st.markdown("""
<div class="info-box">
Cet assistant utilise la technologie <b>RAG (Retrieval Augmented Generation)</b> pour répondre à vos questions
sur les documents PDF. Grâce à l'extraction sémantique avancée, il peut:
<ul>
    <li>Préserver la structure des documents, y compris les tableaux</li>
    <li>Agréger l'information provenant de plusieurs sources</li>
    <li>Filtrer les résultats non pertinents grâce à la similarité cosine</li>
</ul>
</div>
""", unsafe_allow_html=True)

# Création des onglets principaux
tab1, tab2 = st.tabs(["🔍 **Recherche**", "📤 **Gestion des documents**"])

2025-05-18 21:06:45.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:06:45.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:06:45.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:06:45.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:06:45.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:06:45.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:06:45.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [10]:
# Fonction pour charger l'index FAISS existant avec cache
@st.cache_resource
def load_vector_store():
    try:
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        vector_store = FAISS.load_local(
            index_folder, 
            embeddings,
            allow_dangerous_deserialization=True
        )
        return vector_store, True
    except Exception as e:
        st.error(f"Erreur lors du chargement de l'index FAISS: {e}")
        return None, False

# Charger l'index au démarrage
vector_store, success = load_vector_store()

# Fonction pour obtenir les sources disponibles
def get_all_sources(vector_store):
    try:
        all_docs = vector_store.docstore._dict.values()
        sources = set()
        for doc in all_docs:
            source = os.path.basename(doc.metadata.get('source', ''))
            if source:
                sources.add(source)
        return ['Tous les documents'] + sorted(list(sources))
    except:
        return ['Tous les documents']

# Liste étendue des modèles OpenAI
openai_models = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1",
    "o4-mini",          # tout nouveau, successeur d’o3-mini :contentReference[oaicite:5]{index=5}
    "o3",               # grand modèle raisonnement :contentReference[oaicite:6]{index=6}
    "o3-mini",
    "text-embedding-3-small",
    "text-embedding-3-large",

]

2025-05-18 20:53:31.299 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:31.301 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:31.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:31.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:31.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:31.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:31.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [11]:

# Onglet 1: Recherche
with tab1:
    # Paramètres dans la barre latérale
    with st.sidebar:
        st.markdown('<div class="sidebar-section">', unsafe_allow_html=True)
        st.header("⚙️ Paramètres")

        # Nombre de chunks et seuil de similarité
        col1, col2 = st.columns(2)
        with col1:
            k_value = st.slider("Chunks à récupérer", 1, 15, 5)
        with col2:
            similarity_threshold = st.slider("Seuil similarité", 0.0, 1.0, 0.7, 0.05)

        # Température du modèle
        temperature = st.slider("Température du LLM", 0.0, 1.0, 0.1, 0.1, 
                              help="Contrôle la créativité du modèle. Plus élevé = plus créatif mais moins précis.")

        # Choix du modèle (liste étendue)
        llm_model = st.selectbox(
            "Modèle LLM",
            openai_models,
            index=0,
            help="Sélectionnez le modèle OpenAI à utiliser pour la génération de réponses."
        )

        # Option pour l'agrégation
        enable_aggregation = st.checkbox("Activer l'agrégation avancée", True,
                                      help="Utilise map_reduce pour synthétiser l'information de plusieurs sources")
        st.markdown('</div>', unsafe_allow_html=True)

        # Filtrage par sources si l'index est chargé
        if success:
            st.markdown('<div class="sidebar-section">', unsafe_allow_html=True)
            st.header("🔍 Filtrage")
            all_sources = get_all_sources(vector_store)
            selected_sources = st.multiselect(
                "Filtrer par documents",
                options=all_sources,
                default=["Tous les documents"]
            )
            st.markdown('</div>', unsafe_allow_html=True)
        else:
            selected_sources = ["Tous les documents"]

    # Interface utilisateur du chat
    st.markdown('<div class="sub-header">💬 Posez votre question</div>', unsafe_allow_html=True)

    # Zone de saisie pour la question
    query = st.text_area("Votre question sur les documents:", height=100)

    # Ajouter des exemples de questions
    with st.expander("Exemples de questions"):
        example_queries = [
            "Qu'est-ce que le RAG?",
            "Quels sont les avantages des systèmes RAG par rapport aux modèles classiques?",
            "Comment fonctionnent les embeddings dans un système RAG?",
            "Quelles sont les sources d'énergies renouvelables mentionnées dans le document?",
            "Quel est le plan d'investissement de TotalEnergies pour 2023-2025?"
        ]
        cols = st.columns(3)
        for i, example in enumerate(example_queries):
            if cols[i % 3].button(example, key=f"example_{i}"):
                query = example

    # Option pour afficher la visualisation
    show_visualization = st.checkbox("Afficher la visualisation des chunks", False)

    # Bouton pour soumettre la requête
    submit_button = st.button("🔍 Rechercher", type="primary", use_container_width=True)

2025-05-18 20:53:36.510 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.510 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 20:53:36.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [12]:
# Fonction pour interroger le RAG avec similarité cosine améliorée
def query_rag(query):
    # Créer le LLM
    llm = ChatOpenAI(
        model=llm_model,
        temperature=temperature,
    )
    
    # Créer le retriever avec seuil de similarité
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": k_value,
            "score_threshold": similarity_threshold
        }
    )
    
    # Filtrer par source si spécifié
    if selected_sources and 'Tous les documents' not in selected_sources:
        # Créer une fonction de filtrage pour les sources sélectionnées
        def filter_by_sources(docs):
            return [
                doc for doc in docs 
                if os.path.basename(doc.metadata.get('source', '')) in selected_sources
            ]
        
        # Appliquer le filtre au retriever
        retriever.search_kwargs["filter_fn"] = filter_by_sources
    
    # Template de prompt amélioré pour la chaîne stuff
    prompt_template = """
    Tu es un assistant d'information spécialisé qui aide à répondre aux questions basées sur des documents PDF.
    
    Utilise UNIQUEMENT les informations fournies dans le contexte ci-dessous pour répondre.
    Si l'information n'est pas présente dans le contexte, dis simplement "Je ne trouve pas cette information dans les documents fournis."
    Ne fabrique pas de réponse.
    
    IMPORTANT: 
    - Synthétise les informations provenant de sources multiples si nécessaire.
    - Si des informations contradictoires apparaissent, signale-le et présente les différentes perspectives.
    - Pour les informations numériques ou factuelles, cite précisément les sources avec le numéro de page.
    - Si des tableaux sont mentionnés dans les chunks, présente les données sous forme structurée.
    - Mentionne toujours le numéro de page exact dans tes citations.
    
    QUESTION: {question}
    
    CONTEXTE:
    {context}
    
    RÉPONSE (synthétise les informations de manière cohérente, avec citations des sources et numéros de page):
    """
    
    # Templates spécifiques pour map-reduce
    map_template = """
    Tu es un assistant d'information spécialisé qui aide à répondre aux questions basées sur des documents PDF.
    
    Résume le contexte suivant pour répondre à la question.
    
    QUESTION: {question}
    
    CONTEXTE:
    {context}
    
    RÉSUMÉ (ne réponds pas encore à la question, fais uniquement un résumé des informations pertinentes):
    """
    
    reduce_template = """
    Tu es un assistant d'information spécialisé qui aide à répondre aux questions basées sur des documents PDF.
    
    Utilise UNIQUEMENT les informations fournies dans les résumés ci-dessous pour répondre.
    Si l'information n'est pas présente dans les résumés, dis simplement "Je ne trouve pas cette information dans les documents fournis."
    Ne fabrique pas de réponse.
    
    IMPORTANT: 
    - Synthétise les informations provenant de sources multiples si nécessaire.
    - Si des informations contradictoires apparaissent, signale-le et présente les différentes perspectives.
    - Pour les informations numériques ou factuelles, cite précisément les sources, si mentionnées dans les résumés.
    - Si des tableaux sont mentionnés, présente les données sous forme structurée.
    
    QUESTION: {question}
    
    RÉSUMÉS:
    {summaries}
    
    RÉPONSE (synthétise les informations de manière cohérente, avec citations des sources si disponibles):
    """
    
    STUFF_PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    
    MAP_PROMPT = PromptTemplate(
        template=map_template,
        input_variables=["context", "question"]
    )
    
    REDUCE_PROMPT = PromptTemplate(
        template=reduce_template,
        input_variables=["summaries", "question"]
    )
    
    # Créer la chaîne RAG avec capacité d'agrégation
    if enable_aggregation:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="map_reduce",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={
                "question_prompt": MAP_PROMPT,
                "combine_prompt": REDUCE_PROMPT
            }
        )
    else:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": STUFF_PROMPT}
        )
    
    # Exécuter la requête
    result = qa_chain({"query": query})
    
    return {
        "answer": result["result"],
        "source_docs": result["source_documents"]
    }

In [21]:
# Traiter la requête si une question est posée et le bouton est cliqué
if query and submit_button:
    if success:
        with st.status("🔍 Recherche d'informations dans les documents...") as status:
            start_time = time.time()
            
            try:
                # Exécuter la requête
                result = query_rag(query)
                elapsed_time = time.time() - start_time
                status.update(label="✅ Recherche terminée!", state="complete")
            except Exception as e:
                st.error(f"Erreur lors de la recherche: {str(e)}")
                import traceback
                st.error(traceback.format_exc())
                st.stop()
            
            # Afficher la réponse
            st.markdown('<div class="sub-header">📝 Réponse</div>', unsafe_allow_html=True)
            st.markdown(f'<div class="result-container">{result["answer"]}</div>', unsafe_allow_html=True)
            
            # Afficher des métriques
            col1, col2, col3 = st.columns(3)
            with col1:
                st.markdown('<div class="metric-container">', unsafe_allow_html=True)
                st.markdown(f'<div class="metric-value">{elapsed_time:.2f}s</div>', unsafe_allow_html=True)
                st.markdown('<div class="metric-label">Temps de réponse</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
                
            with col2:
                st.markdown('<div class="metric-container">', unsafe_allow_html=True)
                st.markdown(f'<div class="metric-value">{len(set([os.path.basename(doc.metadata.get("source", "")) for doc in result["source_docs"]]))}</div>', unsafe_allow_html=True)
                st.markdown('<div class="metric-label">Documents utilisés</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
                
            with col3:
                st.markdown('<div class="metric-container">', unsafe_allow_html=True)
                st.markdown(f'<div class="metric-value">{len(result["source_docs"])}</div>', unsafe_allow_html=True)
                st.markdown('<div class="metric-label">Chunks utilisés</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
            
            # Afficher les sources
            st.markdown('<div class="sub-header">📄 Sources utilisées</div>', unsafe_allow_html=True)
            source_docs = result["source_docs"]
            
            if source_docs:
                sources_data = []
                for i, doc in enumerate(source_docs):
                    source_name = os.path.basename(doc.metadata.get('source', 'Inconnu'))
                    page_num = None
                    import re
                    if 'page' in doc.metadata:
                        page_num = doc.metadata['page']
                    elif 'header1' in doc.metadata and 'Page ' in doc.metadata['header1']:
                        page_num = doc.metadata['header1'].replace('Page ', '')
                    elif 'header1' in doc.metadata:
                        match = re.search(r'Page (\d+)', doc.metadata['header1'])
                        if match:
                            page_num = match.group(1)
                    if page_num is None:
                        match = re.search(r'# Page (\d+)', doc.page_content)
                        page_num = match.group(1) if match else 'N/A'
                    
                    context = doc.metadata.get('header1', '') + ' > ' + doc.metadata.get('header2', '')
                    similarity = doc.metadata.get('score', 'N/A') if 'score' in doc.metadata else 'N/A'
                    
                    sources_data.append({
                        "N°": i+1,
                        "Document": source_name,
                        "Page": page_num,
                        "Contexte": context,
                        "Pertinence": similarity if isinstance(similarity, float) else 'N/A',
                        "Extrait": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content
                    })
                
                st.markdown('<div class="dataframe-container">', unsafe_allow_html=True)
                st.dataframe(
                    pd.DataFrame(sources_data),
                    use_container_width=True,
                    column_config={
                        "N°": st.column_config.NumberColumn("N°", width="small"),
                        "Document": st.column_config.TextColumn("Document", width="medium"),
                        "Page": st.column_config.TextColumn("Page", width="small"),
                        "Contexte": st.column_config.TextColumn("Contexte", width="medium"),
                        "Extrait": st.column_config.TextColumn("Extrait", width="large"),
                        "Pertinence": st.column_config.ProgressColumn(
                            "Pertinence", 
                            min_value=0, 
                            max_value=1,
                            format="%.2f"
                        ) if any(isinstance(x.get("Pertinence"), float) for x in sources_data) else None
                    }
                )
                st.markdown('</div>', unsafe_allow_html=True)
                
                if show_visualization and len(source_docs) > 1:
                    try:
                        st.markdown('<div class="sub-header">📊 Visualisation des chunks</div>', unsafe_allow_html=True)
                        viz_data = pd.DataFrame(sources_data)
                        st.subheader("Distribution des chunks par document")
                        doc_counts = viz_data["Document"].value_counts()
                        fig, ax = plt.subplots(figsize=(8, 5))
                        doc_counts.plot(kind='pie', autopct='%1.1f%%', ax=ax)
                        plt.ylabel('')
                        st.pyplot(fig)
                        
                        pertinence_cols = [col for col in viz_data.columns if 'Pertinence' in col]
                        if pertinence_cols and viz_data[pertinence_cols[0]].apply(lambda x: isinstance(x, (int, float))).any():
                            st.subheader("Pertinence des chunks récupérés")
                            fig, ax = plt.subplots(figsize=(10, 6))
                            sns.barplot(x="N°", y=pertinence_cols[0], data=viz_data, palette="viridis", ax=ax)
                            ax.set_title("Score de similarité par chunk")
                            ax.set_ylabel("Score de similarité")
                            ax.set_xlabel("Numéro du chunk")
                            plt.tight_layout()
                            st.pyplot(fig)
                    except Exception as e:
                        st.info(f"Impossible de générer la visualisation. Erreur: {str(e)}")
            else:
                st.info("Aucune source pertinente trouvée. Essayez d'ajuster le seuil de similarité ou les paramètres de filtrage.")
            
            st.markdown("### Évaluation de la réponse")
            feedback = st.radio(
                "Cette réponse était-elle utile?",
                ["Très utile", "Partiellement utile", "Pas utile"]
            )
            feedback_text = st.text_area("Commentaires supplémentaires (facultatif):", height=100, key="feedback")
            if st.button("Envoyer l'évaluation"):
                st.success("Merci pour votre feedback! Il nous aidera à améliorer le système.")
    else:
        st.error("Impossible de charger l'index FAISS. Veuillez vérifier que l'index existe et est accessible.")



In [22]:
# Onglet 2: Gestion des documents
with tab2:
    st.markdown('<div class="sub-header">📤 Ajouter de nouveaux documents</div>', unsafe_allow_html=True)
    
    # Section améliorée pour l'upload de fichiers
    st.markdown("""
    <div class="info-box">
    Cette section vous permet d'ajouter de nouveaux documents PDF à l'index. Les fichiers seront automatiquement traités,
    découpés en chunks et indexés pour être disponibles dans vos recherches.
    </div>
    """, unsafe_allow_html=True)
    
    # Zone d'upload bien visible
    uploaded_files = st.file_uploader(
        "Téléchargez vos fichiers PDF",
        type=['pdf'],
        accept_multiple_files=True,
        help="Vous pouvez télécharger plusieurs fichiers PDF simultanément"
    )
    
    # Paramètres de traitement des PDF
    with st.expander("Paramètres avancés de traitement"):
        col1, col2 = st.columns(2)
        with col1:
            chunk_size = st.slider("Taille des chunks", 200, 1000, 350, 50,
                                help="Plus petite taille = plus de précision, plus grande taille = plus de contexte")
        with col2:
            chunk_overlap = st.slider("Chevauchement", 0, 200, 50, 10,
                                    help="Contrôle le chevauchement entre les chunks pour préserver le contexte")
        
        embedding_model = st.selectbox(
            "Modèle d'embedding",
            ["text-embedding-3-small", "text-embedding-3-large"],
            index=0,
            help="Modèle utilisé pour créer les embeddings vectoriels"
        )

2025-05-18 21:19:17.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.145 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.149 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:19:17.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [23]:
    # Fonction pour traiter les nouveaux fichiers
    def process_uploaded_files(uploaded_files, vector_store):
        if not uploaded_files:
            return vector_store, 0
        
        # Créer un dossier temporaire pour les fichiers
        temp_dir = tempfile.mkdtemp()
        
        # Enregistrer les fichiers téléchargés
        pdf_paths = []
        for uploaded_file in uploaded_files:
            file_path = os.path.join(temp_dir, uploaded_file.name)
            with open(file_path, "wb") as f:
                f.write(uploaded_file.getbuffer())
            pdf_paths.append(file_path)
        
        # Initialiser le text splitter pour un chunking plus fin
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        
        # Traiter chaque fichier
        new_chunks = []
        for pdf_path in pdf_paths:
            try:
                # Utiliser PyMuPDF pour extraire texte et tableaux
                doc = fitz.open(pdf_path)
                pdf_text = ""
                
                doc_metadata = {
                    "source": pdf_path,
                    "title": os.path.basename(pdf_path),
                    "total_pages": len(doc)
                }
                
                for page_num, page in enumerate(doc):
                    # Extraire le texte
                    text = page.get_text("text")
                    
                    # Extraire les tableaux en format texte structuré
                    tables = page.get_text("blocks")
                    table_text = ""
                    for block in tables:
                        if len(block) > 6 and block[6] == 1:  # Type 1 = table
                            table_text += f"\nTableau: {block[4]}\n"
                    
                    # Extraire et décrire les images
                    image_info = ""
                    for img in page.get_images(full=True):
                        image_info += f"\n[Image: {img[0]}]\n"
                    
                    # Combiner avec des marqueurs de structure
                    page_content = f"# Page {page_num+1}\n{text}\n{table_text}\n{image_info}"
                    pdf_text += page_content
                
                # Découper avec préservation de la structure
                try:
                    header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
                        ("#", "header1"),
                        ("##", "header2"),
                    ])
                    md_header_splits = header_splitter.split_text(pdf_text)
                    
                    # Découper en chunks tout en préservant les métadonnées hiérarchiques
                    chunks = []
                    for split in md_header_splits:
                        # Préserver le contexte hiérarchique dans les métadonnées
                        split_metadata = {**doc_metadata, **split.metadata}
                        # Découper davantage si nécessaire
                        smaller_chunks = text_splitter.split_text(split.page_content)
                        # Créer des Documents avec contexte préservé
                        for chunk in smaller_chunks:
                            chunks.append(Document(page_content=chunk, metadata=split_metadata))
                except Exception as e:
                    # Fallback si le découpage hiérarchique échoue
                    chunks = []
                    pages = [Document(page_content=page.get_text("text"), 
                                     metadata={"source": pdf_path, "page": i+1}) 
                            for i, page in enumerate(doc)]
                    chunks = text_splitter.split_documents(pages)
                
                new_chunks.extend(chunks)
            except Exception as e:
                st.error(f"Erreur avec {os.path.basename(pdf_path)}: {str(e)}")
        
        # Créer les embeddings et ajouter à l'index
        if new_chunks:
            try:
                # S'assurer que l'instance d'embeddings est disponible
                embeddings = OpenAIEmbeddings(model=embedding_model)
                
                # Ajouter directement les documents à FAISS
                vector_store.add_documents(new_chunks)
                
                # Sauvegarde de l'index FAISS mis à jour
                vector_store.save_local(index_folder)
                
                # Nettoyer les fichiers temporaires
                shutil.rmtree(temp_dir)
                
                return vector_store, len(new_chunks)
            except Exception as e:
                st.error(f"Erreur lors de la mise à jour de l'index: {str(e)}")
        
        # Nettoyer les fichiers temporaires
        shutil.rmtree(temp_dir)
        
        return vector_store, 0

In [24]:


    # Bouton pour déclencher le traitement avec barre de progression
    if uploaded_files:
        if st.button("📥 Traiter et indexer les fichiers", type="primary", use_container_width=True):
            with st.spinner("Traitement des fichiers en cours..."):
                progress_bar = st.progress(0)
                total_files = len(uploaded_files)
                
                for i, _ in enumerate(uploaded_files):
                    progress_bar.progress((i + 1) / total_files)
                
                vector_store, num_added = process_uploaded_files(uploaded_files, vector_store)
                
                if num_added > 0:
                    st.success(f"✅ {num_added} nouveaux chunks ont été ajoutés à l'index à partir de {len(uploaded_files)} fichiers!")
                    # Rafraîchir les données
                    st.cache_data.clear()
                    st.experimental_rerun()
                else:
                    st.error("❌ Aucun chunk n'a pu être ajouté. Vérifiez les fichiers et réessayez.")
    
    # Afficher les statistiques sur l'index actuel
    st.markdown('<div class="sub-header">📊 Statistiques de l\'index</div>', unsafe_allow_html=True)
    
    if success:
        try:
            # Récupérer les statistiques
            all_docs = vector_store.docstore._dict.values()
            total_chunks = len(vector_store.index_to_docstore_id)
            
            # Analyser les documents
            sources_dict = {}
            pages_dict = {}
            
            for doc in all_docs:
                source = os.path.basename(doc.metadata.get('source', 'Inconnu'))
                if source in sources_dict:
                    sources_dict[source] += 1
                else:
                    sources_dict[source] = 1
                
                page = doc.metadata.get('page', 'N/A')
                if source in pages_dict:
                    if page not in pages_dict[source]:
                        pages_dict[source].append(page)
                else:
                    pages_dict[source] = [page]
            
            # Afficher les statistiques générales
            col1, col2, col3 = st.columns(3)
            with col1:
                st.markdown('<div class="metric-container">', unsafe_allow_html=True)
                st.markdown(f'<div class="metric-value">{total_chunks}</div>', unsafe_allow_html=True)
                st.markdown('<div class="metric-label">Chunks totaux</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
                
            with col2:
                st.markdown('<div class="metric-container">', unsafe_allow_html=True)
                st.markdown(f'<div class="metric-value">{len(sources_dict)}</div>', unsafe_allow_html=True)
                st.markdown('<div class="metric-label">Documents</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
                
            with col3:
                total_pages = sum(len(pages) for pages in pages_dict.values())
                st.markdown('<div class="metric-container">', unsafe_allow_html=True)
                st.markdown(f'<div class="metric-value">{total_pages}</div>', unsafe_allow_html=True)
                st.markdown('<div class="metric-label">Pages</div>', unsafe_allow_html=True)
                st.markdown('</div>', unsafe_allow_html=True)
            
            # Visualiser la répartition des chunks
            if sources_dict:
                st.subheader("Répartition des chunks par document")
                
                # Créer un dataframe pour la visualisation
                df_sources = pd.DataFrame({
                    'Document': list(sources_dict.keys()),
                    'Nombre de chunks': list(sources_dict.values())
                })
                
                # Trier par nombre de chunks
                df_sources = df_sources.sort_values('Nombre de chunks', ascending=False)
                
                # Créer le graphique
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.barplot(x='Document', y='Nombre de chunks', data=df_sources, palette='viridis', ax=ax)
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                st.pyplot(fig)
                
                # Afficher le tableau détaillé
                st.subheader("Détails des documents indexés")
                
                # Préparer les données pour le tableau
                table_data = []
                for source, chunks in sources_dict.items():
                    pages = len(pages_dict.get(source, []))
                    table_data.append({
                        "Document": source,
                        "Nombre de chunks": chunks,
                        "Nombre de pages": pages,
                        "Chunks/page": round(chunks / max(1, pages), 2)
                    })
                
                # Afficher le tableau
                st.markdown('<div class="dataframe-container">', unsafe_allow_html=True)
                st.dataframe(
                    pd.DataFrame(table_data),
                    use_container_width=True,
                    hide_index=True
                )
                st.markdown('</div>', unsafe_allow_html=True)
                
        except Exception as e:
            st.error(f"Erreur lors de la récupération des statistiques: {str(e)}")
    else:
        st.error("Index FAISS non chargé. Impossible d'afficher les statistiques.")
    
    # Option pour réinitialiser l'index
    with st.expander("⚠️ Réinitialiser l'index"):
        st.warning("Attention: Cette opération supprimera définitivement l'index FAISS actuel et toutes les données qu'il contient.")
        if st.button("🗑️ Réinitialiser l'index", use_container_width=True):
            confirm = st.text_input("Tapez 'CONFIRMER' pour réinitialiser l'index", key="confirm_reset")
            if confirm == "CONFIRMER":
                try:
                    # Supprimer l'index
                    if os.path.exists(os.path.join(index_folder, "index.faiss")):
                        os.remove(os.path.join(index_folder, "index.faiss"))
                    if os.path.exists(os.path.join(index_folder, "index.pkl")):
                        os.remove(os.path.join(index_folder, "index.pkl"))
                    
                    st.success("✅ Index réinitialisé avec succès! L'application va redémarrer.")
                    st.cache_data.clear()
                    st.cache_resource.clear()
                    st.experimental_rerun()
                except Exception as e:
                    st.error(f"Erreur lors de la réinitialisation: {str(e)}")

2025-05-18 21:21:37.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:37.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
# Pied de page
st.markdown("---")
col1, col2 = st.columns([3, 1])
with col1:
    st.markdown("Développé avec RAG avancé et Streamlit")
with col2:
    st.markdown("Version: 3.0")

2025-05-18 21:21:41.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.525 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-18 21:21:41.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar